## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [9]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [10]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [11]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
she-he
herself-himself
her-his
woman-man
daughter-son
businesswoman-businessman
girl-boy
actress-actor
chairwoman-chairman
heroine-hero
mother-father
spokeswoman-spokesman
sister-brother
girls-boys
sisters-brothers
queen-king
niece-nephew
councilwoman-councilman
motherhood-fatherhood
women-men
petite-lanky
ovarian_cancer-prostate_cancer
Anne-John
schoolgirl-schoolboy
granddaughter-grandson
aunt-uncle
matriarch-patriarch
twin_sister-twin_brother
mom-dad
lesbian-gay
husband-younger_brother
gal-dude
lady-gentleman
sorority-fraternity
mothers-fathers
grandmother-grandfather
blouse-shirt
soprano-baritone
queens-kings
Jill-Greg
daughters-sons
grandma-grandpa
volleyball-football
diva-superstar
mommy-kid
Sarah-Matthew
hairdresser-barber
softball-baseball
goddess-god
Aisha-Jamal
waitress-waiter
princess-prince
filly-colt
mare-gelding
ladies-gentlemen
childhood-boyhood
interior_designer-architect
nun-priest
wig-beard
granddaughters-grandso

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [12]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.2075867, 'skipper'),
  (-0.20267203, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492391, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.17289859, 'captain'),
  (-0.16785556, 'architect'),
  (-0.16702037, 'financier'),
  (-0.1631364, 'warrior'),
  (-0.15280864, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637241, 'magician'),
  (-0.14401692, 'fighter_pilot'),
  (-0.13782284, 'boss'),
  (-0.13718201, 'industrialist'),
  (-0.13684887, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560377, 'stylist'),
  (0.22363171, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125953, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946159, 'paralegal'),
  (0.25276467, 'ballerina'),
  (0.2571882, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317625, 'receptionist'),
  (0.2754029, 'waitress'),
  (

In [13]:
sp

[(-0.23798442, 'maestro'),
 (-0.21665451, 'statesman'),
 (-0.2075867, 'skipper'),
 (-0.20267203, 'protege'),
 (-0.2020676, 'businessman'),
 (-0.19492391, 'sportsman'),
 (-0.18836352, 'philosopher'),
 (-0.1807366, 'marksman'),
 (-0.17289859, 'captain'),
 (-0.16785556, 'architect'),
 (-0.16702037, 'financier'),
 (-0.1631364, 'warrior'),
 (-0.15280864, 'major_leaguer'),
 (-0.15001445, 'trumpeter'),
 (-0.14718868, 'broadcaster'),
 (-0.14637241, 'magician'),
 (-0.14401692, 'fighter_pilot'),
 (-0.13782284, 'boss'),
 (-0.13718201, 'industrialist'),
 (-0.13684887, 'pundit'),
 (-0.13590316, 'lieutenant'),
 (-0.13587037, 'mechanic'),
 (-0.1306604, 'understudy'),
 (-0.1257944, 'gangster'),
 (-0.12360984, 'coach'),
 (-0.12276026, 'commander'),
 (-0.12088909, 'ballplayer'),
 (-0.12061268, 'hitman'),
 (-0.11973348, 'baron'),
 (-0.11844293, 'mathematician'),
 (-0.11563749, 'soft_spoken'),
 (-0.11345602, 'barber'),
 (-0.11345409, 'salesman'),
 (-0.11112277, 'carpenter'),
 (-0.10962007, 'commentator'),

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [14]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [15]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [16]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

for (a,b,c) in a_racial:
    print(a+"-"+b)

Sarah-Keisha
defensemen-cornerbacks
hipster-hip_hop
punter-cornerback
singer_songwriter-rapper
defenseman-defensive_tackle
pole_vault-triple_jump
musicians-artistes
musician-artiste
catcher-wide_receiver
rock_n_roll-reggae
kicker-kick_returner
tavern-barbershop
freestyle_relay-meter_hurdles
lefthander-swingman
bacon-fried_chicken
artists-rappers
equipment-equipments
hockey-basketball
wool-cotton
unassisted_goal-layup
chocolates-sweets
buddy-cousin
priest-preacher
blue-black
medley_relay-meter_dash
quirky-funky
rabbi-imam
grapes-mango
telecommunications-telecommunication
pitchers-defensive_linemen
passages-verses
er-o
acoustic-soulful
punting-punt_returns
thefts-armed_robbery
bar-nightclub
digs-rebounds
Greg-Geoffrey
cellist-saxophonist
smarts-quickness
puck-halfcourt
quarterback-tailback
fox-leopard
pedophiles-rapists
potatoes-flour
en-el
infrastructure-infrastructural
evangelism-gospel
fiance-aunt
pointers-dunks
baseman-defensive_lineman
pedophile-rapist
joked-smiled
beer-soft_drink
g

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [17]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.3154624, 'artiste'),
  (-0.27369624, 'shopkeeper'),
  (-0.27285585, 'taxi_driver'),
  (-0.24248752, 'cab_driver'),
  (-0.230962, 'preacher'),
  (-0.21709058, 'boxer'),
  (-0.20973533, 'laborer'),
  (-0.20361684, 'barber'),
  (-0.19625023, 'cleric'),
  (-0.182731, 'bodyguard'),
  (-0.18250425, 'gangster'),
  (-0.1816296, 'singer'),
  (-0.16877083, 'maid'),
  (-0.16871038, 'entertainer'),
  (-0.16197538, 'cabbie'),
  (-0.15332887, 'housewife'),
  (-0.14839593, 'civil_servant'),
  (-0.14115773, 'policeman'),
  (-0.13648951, 'minister'),
  (-0.1329656, 'drug_addict')],
 [(0.087792546, 'organist'),
  (0.09007451, 'philanthropist'),
  (0.09135293, 'cinematographer'),
  (0.093180254, 'manager'),
  (0.09358391, 'investment_banker'),
  (0.096878394, 'professor_emeritus'),
  (0.097828984, 'curator'),
  (0.09864862, 'freelance_writer'),
  (0.09917156, 'programmer'),
  (0.10142014, 'screenwriter'),
  (0.10198846, 'author'),
  (0.10438656, 'inventor'),
  (0.106778406, 'adventurer'),
  (0.1096

In [18]:
sp

[(-0.3154624, 'artiste'),
 (-0.27369624, 'shopkeeper'),
 (-0.27285585, 'taxi_driver'),
 (-0.24248752, 'cab_driver'),
 (-0.230962, 'preacher'),
 (-0.21709058, 'boxer'),
 (-0.20973533, 'laborer'),
 (-0.20361684, 'barber'),
 (-0.19625023, 'cleric'),
 (-0.182731, 'bodyguard'),
 (-0.18250425, 'gangster'),
 (-0.1816296, 'singer'),
 (-0.16877083, 'maid'),
 (-0.16871038, 'entertainer'),
 (-0.16197538, 'cabbie'),
 (-0.15332887, 'housewife'),
 (-0.14839593, 'civil_servant'),
 (-0.14115773, 'policeman'),
 (-0.13648951, 'minister'),
 (-0.1329656, 'drug_addict'),
 (-0.13115542, 'magistrate'),
 (-0.12681912, 'hairdresser'),
 (-0.1256297, 'dancer'),
 (-0.12385333, 'janitor'),
 (-0.1232332, 'footballer'),
 (-0.12240437, 'envoy'),
 (-0.120815866, 'custodian'),
 (-0.119047984, 'parliamentarian'),
 (-0.117543384, 'teenager'),
 (-0.113934465, 'saxophonist'),
 (-0.11218839, 'lawmaker'),
 (-0.11134215, 'diplomat'),
 (-0.11063444, 'constable'),
 (-0.109942526, 'crooner'),
 (-0.10557103, 'caretaker'),
 (-0.10

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [19]:
from debiaswe.debias import debias

In [20]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [21]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('SONS', 'DAUGHTERS'), ('COLT', 'FILLY'), ('grandfather', 'grandmother'), ('prince', 'princess'), ('Fella', 'Granny'), ('Gelding', 'Mare'), ('MONASTERY', 'CONVENT'), ('Gentlemen', 'Ladies'), ('grandson', 'granddaughter'), ('fella', 'granny'), ('FATHERHOOD', 'MOTHERHOOD'), ('Testosterone', 'Estrogen'), ('males', 'females'), ('BROTHERS', 'SISTERS'), ('Fathers', 'Mothers'), ('HIS', 'HER'), ('He', 'She'), ('FRATERNITY', 'SORORITY'), ('SPOKESMAN', 'SPOKESWOMAN'), ('Boy', 'Girl'), ('Schoolboy', 'Schoolgirl'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('congressman', 'congresswoman'), ('fraternity', 'sorority'), ('dudes', 'gals'), ('Prince', 'Princess'), ('Uncle', 'Aunt'), ('Catholic_Priest', 'Nun'), ('boys', 'girls'), ('men', 'women'), ('brother', 'sister'), ('fathers', 'mothers'), ('Gentleman', 'Lady'), ('father', 'mother'), ('businessman', 'businesswoman'), ('Chairman', 'Chairwoman'), ('Father', 'Mother'), ('Colt', 'Fill

In [22]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.4067585, 'businessman'),
  (-0.32398775, 'councilman'),
  (-0.3096709, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.07300486, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.04946825, 'minister'),
  (-0.043583877, 'archbishop'),
  (-0.040207203, 'bishop'),
  (-0.038332477, 'commissioner'),
  (-0.035724394, 'surgeon'),
  (-0.033134, 'trader'),
  (-0.032377206, 'observer'),
  (-0.032095857, 'neurosurgeon'),
  (-0.031450115, 'priest'),
  (-0.03113396, 'skipper'),
  (-0.029659173, 'lawmaker'),
  (-0.029511211, 'commander')],
 [(0.02996565, 'teenager'),
  (0.030237038, 'instructor'),
  (0.030946158, 'student'),
  (0.031116989, 'paralegal'),
  (0.032039426, 'bookkeeper'),
  (0.032434613, 'cinematographer'),
  (0.03432907, 'graphic_designer'),
  (0.03470566, 'lifeguard'),
  (0.03566653, 'janitor'),
  (0.035971925, 'drummer'),
  (0.042120147, 'wrestler'),
  (0.043902267, 'hairdresser'),
  (0.048133135, 'firefighter'),
  (0.23776129,

In [23]:
sp

[(-0.3154624, 'artiste'),
 (-0.27369624, 'shopkeeper'),
 (-0.27285585, 'taxi_driver'),
 (-0.24248752, 'cab_driver'),
 (-0.230962, 'preacher'),
 (-0.21709058, 'boxer'),
 (-0.20973533, 'laborer'),
 (-0.20361684, 'barber'),
 (-0.19625023, 'cleric'),
 (-0.182731, 'bodyguard'),
 (-0.18250425, 'gangster'),
 (-0.1816296, 'singer'),
 (-0.16877083, 'maid'),
 (-0.16871038, 'entertainer'),
 (-0.16197538, 'cabbie'),
 (-0.15332887, 'housewife'),
 (-0.14839593, 'civil_servant'),
 (-0.14115773, 'policeman'),
 (-0.13648951, 'minister'),
 (-0.1329656, 'drug_addict'),
 (-0.13115542, 'magistrate'),
 (-0.12681912, 'hairdresser'),
 (-0.1256297, 'dancer'),
 (-0.12385333, 'janitor'),
 (-0.1232332, 'footballer'),
 (-0.12240437, 'envoy'),
 (-0.120815866, 'custodian'),
 (-0.119047984, 'parliamentarian'),
 (-0.117543384, 'teenager'),
 (-0.113934465, 'saxophonist'),
 (-0.11218839, 'lawmaker'),
 (-0.11134215, 'diplomat'),
 (-0.11063444, 'constable'),
 (-0.109942526, 'crooner'),
 (-0.10557103, 'caretaker'),
 (-0.10

In [24]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
girl-boy
female-male
girls-boys
females-males
princess-prince
councilwoman-councilman
daughter-son
chairwoman-chairman
gals-dudes
mothers-fathers
she-he
queen-king
mare-gelding
estrogen-testosterone
woman-man
daughters-sons
grandmother-grandfather
women-men
convent-monastery
sorority-fraternity
ovarian_cancer-prostate_cancer
grandma-grandpa
husbands-wives
twin_sister-twin_brother
sister-brother
sisters-brothers
herself-himself
her-his
niece-nephew
ladies-gentlemen
schoolgirl-schoolboy
businesswoman-businessman
congresswoman-congressman
motherhood-fatherhood
ex_boyfriend-ex_girlfriend
aunt-uncle
mother-father
mom-dad
granddaughter-grandson
moms-dads
spokeswoman-spokesman
granddaughters-grandsons
filly-colt
queens-kings
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-

In [25]:
for w in profession_words:
    #print(w,"\t",E.v(w))
    print(w)

accountant
acquaintance
actor
actress
adjunct_professor
administrator
adventurer
advocate
aide
alderman
alter_ego
ambassador
analyst
anthropologist
archaeologist
archbishop
architect
artist
artiste
assassin
assistant_professor
associate_dean
associate_professor
astronaut
astronomer
athlete
athletic_director
attorney
author
baker
ballerina
ballplayer
banker
barber
baron
barrister
bartender
biologist
bishop
bodyguard
bookkeeper
boss
boxer
broadcaster
broker
bureaucrat
businessman
businesswoman
butcher
butler
cab_driver
cabbie
cameraman
campaigner
captain
cardiologist
caretaker
carpenter
cartoonist
cellist
chancellor
chaplain
character
chef
chemist
choreographer
cinematographer
citizen
civil_servant
cleric
clerk
coach
collector
colonel
columnist
comedian
comic
commander
commentator
commissioner
composer
conductor
confesses
congressman
constable
consultant
cop
correspondent
councilman
councilor
counselor
critic
crooner
crusader
curator
custodian
dad
dancer
dean
dentist
deputy
dermatologist

In [26]:
import pickle
with open('biased_embed.pkl', 'rb') as f:
    word_embed_biased = pickle.load(f)
    
with open('vocab.pkl', 'rb') as f:
    word_vocab_biased = pickle.load(f)

In [52]:
for w in profession_words:
    i=word_vocab_biased.index(w)
    word_embed_biased[i] = list(E.v(w))

In [51]:
word_embed_biased[word_vocab_biased.index("dermatologist")]

['-0.042102',
 '0.0459709',
 '0.0446054',
 '-0.0350471',
 '-0.044833',
 '0.14474',
 '-0.0402814',
 '-0.122892',
 '-0.0295852',
 '-0.0755561',
 '-0.00364126',
 '-0.0110945',
 '0.0395987',
 '0.0641772',
 '0.037778',
 '0.150202',
 '0.0316334',
 '0.101955',
 '-0.0209372',
 '0.0623565',
 '0.00250336',
 '0.0064291',
 '-0.0150202',
 '-0.0523431',
 '-0.106507',
 '-1.23901e-05',
 '0.078287',
 '0.0228717',
 '-0.0710045',
 '-0.0355023',
 '-0.0600808',
 '-0.00870488',
 '-0.0559843',
 '-0.0393711',
 '-0.0559843',
 '-0.0737355',
 '0.105596',
 '0.0541637',
 '0.0368677',
 '-0.0341368',
 '0.0320886',
 '-0.0969485',
 '-0.00364126',
 '-0.0224165',
 '-0.0159305',
 '-0.0357298',
 '0.0555292',
 '-0.0380056',
 '-0.0105824',
 '0.00836351',
 '0.0591704',
 '0.0787422',
 '-0.0452881',
 '-0.015134',
 '-0.042102',
 '-0.0380056',
 '0.013484',
 '0.0641772',
 '-0.118341',
 '-0.0427848',
 '-0.0441503',
 '-0.0562119',
 '0.118341',
 '0.0623565',
 '0.0316334',
 '0.120162',
 '-0.0746458',
 '0.132906',
 '0.0937624',
 '-0.0

In [53]:
word_embed_biased[word_vocab_biased.index("dermatologist")]

[-0.029667238,
 0.059079155,
 0.05182201,
 -0.035950463,
 -0.047159154,
 0.1480636,
 -0.04181541,
 -0.12312425,
 -0.0154370945,
 -0.08124652,
 -0.0018274287,
 -0.01150983,
 0.042070925,
 0.06481417,
 0.02527552,
 0.15130301,
 0.0320727,
 0.11253996,
 -0.019483255,
 0.06231522,
 0.0011908514,
 0.012773615,
 -0.0080630565,
 -0.04495611,
 -0.10847761,
 -0.0054334346,
 0.08165613,
 0.025018154,
 -0.065027736,
 -0.022041926,
 -0.05612206,
 0.003983337,
 -0.043704867,
 -0.031683173,
 -0.047128066,
 -0.061758533,
 0.10792021,
 0.05180075,
 0.042515483,
 -0.029212514,
 0.045137327,
 -0.10019128,
 0.0028444196,
 -0.016936075,
 -0.011739584,
 -0.034032915,
 0.052684963,
 -0.042863727,
 -0.016220469,
 -0.004533073,
 0.053867705,
 0.088868804,
 -0.048841342,
 -0.029619712,
 -0.031513773,
 -0.0360357,
 0.019023744,
 0.067920454,
 -0.1258757,
 -0.04306329,
 -0.046687383,
 -0.044716183,
 0.1195135,
 0.054564316,
 0.034236524,
 0.12172535,
 -0.08441661,
 0.113833584,
 0.09338216,
 -0.061895583,
 0.051

In [54]:
import pickle
with open('debiased_embed.pkl', 'wb') as f:
    pickle.dump(word_embed_biased, f)